# Exercise 5 - Classification Algorithms (BBNN & SVM)

## a) Back Propogation Neural Network

### Program 1 - Implementing BPNN from scratch

#### AIM
To implement Back Propogation Neural Network from scratch in python.

#### ALGORITHM
##### Forward propogation
###### Single input
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\newcommand{\matrix}[1]{\begin{bmatrix}{#1}\end{bmatrix}}
\begin{align*}
\vect{o}_{-1} &= \vect{x}_i\\
\vect{o}_k &= \vect{\phi}_k\left(
\vect{W}_k 
\matrix{ 1 \\ \vect{o}_{k-1}}
\right) &\text{for }k = 0,..,n-1
\end{align*}
$$

###### Batch input
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{align*}
\vect{O}_{-1} &= \vect{X}^T\\
\vect{O}_k &= \vect{\phi}_k\left(
\vect{W}_k^T 
\matrix{ \vect{1} \\ \vect{O}_{k-1}}
\right) &\text{for }k = 0,..,n-1
\end{align*}
$$

##### Error
###### Single Input
$$
E = \frac{1}{2}||\vect{o}_{n-1}-\vect{t}||_2
$$
###### Batch Input
$$
E = \sum_i \frac{1}{2}||\vect{o}^{(i)}_{n-1}-\vect{t}||_2
$$


##### Backward Propogation

###### Single input
$$
\newcommand{\diff}[2]{\frac{\mathrm{d}{#1}}{\mathrm{d}{#2}}}
\vect{\delta}_{k} = \diff{E}{\vect{o}_{k}}\circ \vect{\phi}'(\text{net}_{k} )\\
\diff{E}{\vect{o}_{k}} = \begin{cases}
(\vect{o}_{n-1}-\vect{t}) & k=n-1\\
\vect{W}^T_{k+1}\vect{o}_{k+1} & k = n-2,...,-1 \\
\end{cases}\\
\vect{\phi}'(\text{net}_{k}) = \vect{o}_k \circ (\vect{1}-\vect{o}_k)  \text{ for }   k = n-1,...,0 \\
$$

###### Batch input
$$
\newcommand{\diff}[2]{\frac{\mathrm{d}{#1}}{\mathrm{d}{#2}}}
\vect{\Delta}_{k} = \diff{E}{\vect{O}_{k}}\circ \vect{\phi}'(\text{net}_{k} )\\
\diff{E}{\vect{O}_{k}} = \begin{cases}
(\vect{O}_{n-1}-\vect{t}) & k=n-1\\
\vect{W}_{k+1}^T \vect{O}_{k+1} & k = n-2,...,-1 \\
\end{cases}\\
\vect{\phi}'(\text{net}_{k}) = \vect{O}_k \circ (\vect{1}-\vect{O}_k)  \text{ for }   k = n-1,...,0 \\
$$

##### Weight Update
###### Single Input
$$
\vect{W}_k := \vect{W}_k - \alpha \diff{E}{\vect{W}_k}\\
\diff{E}{\vect{W}_k} = \vect{\delta}_{k}
\matrix{1 \\ \vect{o}_{k-1}}^T\ \text{for } k = n-1,...,0 \\
$$

###### Batch Input
$$
\vect{W}_k := \vect{W}_k - \alpha \diff{E}{\vect{W}_k}\\
\diff{E}{\vect{W}_k} = \vect{\Delta}_{k}
\matrix{1 \\ \vect{O}_{k-1}}^T \text{for } k = n-1,...,0 \\
$$

where  
$\vect{x}_k$ - the input vector  
$\vect{o}_k$ - the output vector of the $k$th layer  
$\vect{\delta}_k$ - the delta vector of the $k$th layer  
$\vect{W}_k$ - the weight matrix of the $k$th layer  
$\vect{X}_k$ - the batch input matfix  
$\vect{O}_k$ - the batch output matrix of the $k$th layer  
$\vect{\Delta}_k$ - the batch delta matrix of the $k$th layer  

#### Part 1 - Defining class for BPNN

In [1]:
import numpy as np
import pandas as pd

In [2]:
class BPNN:
    def __init__(
        self,hidden_layer_sizes,
        l_rate=.001,
        n_epoch=20,
        batch_size=20,
        random_state=0
    ):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.l_rate= l_rate
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.random_state = random_state
        
        self.n = len(hidden_layer_sizes)+1
        self.outputs = [None]*(self.n+1)
        self.delta = [None]*self.n
        
        
    def activation(self,x):
        return 1/(1+np.exp(-x))
    
    def d_activation(self,x):
         return x * (1 - x)
    
    def pad_ones(X):
        pad_width = [(1,0),(0,0)]
        return np.pad(X,pad_width=pad_width,constant_values=1)
    
    def inputs(self,i):
        return BPNN.pad_ones(self.outputs[i-1])
    
    def initialize_random_weights(self):
        np.random.seed(self.random_state)
        self.weights = [
            np.random.random((o,i+1))
            for i,o in zip(
                [self.n_inputs]+self.hidden_layer_sizes,
                self.hidden_layer_sizes+[self.n_outputs]
            )
        ]

    def forward_propagate(self, inputs):
        assert (self.weights is not None),"weights not given"
        self.outputs[-1] = inputs
        for i in range(self.n):
            self.outputs[i] = self.activation(
                self.weights[i] @ self.inputs(i)
            )
        return self.outputs[self.n-1]
    
    def backward_propagate_error(self, expected):
        for i in range(self.n-1, -1, -1):
            if i == self.n-1:
                errors = self.outputs[i] - expected
            else:
                errors = self.weights[i+1][:,1:].T @ self.delta[i+1] 
            self.delta[i] = errors * self.d_activation(self.outputs[i])

    def update_weights(self):
        for i in range(self.n):
            self.weights[i] -= self.l_rate * self.delta[i] @ self.inputs(i).T
        
    def error(self,actual,predicted):
        error = actual-predicted
        return np.sum(error * error)
    
    def fit(self, X_train,y_train,verbose=False):
        m = X_train.shape[0]
        input_matrix = X_train.T
        classes = np.unique(y_train)
        target_matrix = (y_train.reshape(-1,1) == classes).T
        self.n_inputs = input_matrix.shape[0]
        self.n_outputs = target_matrix.shape[0]
        self.initialize_random_weights()
        if verbose:
            print("Initial Weights:")
            print(*self.weights,sep="\n")
            print("Training:")
            
        for epoch in range(self.n_epoch):
            sum_error = 0
            fs = range(m+self.batch_size)
            for f,t in zip(fs,fs[1:]):
                outputs = self.forward_propagate(input_matrix[:,f:t])
                sum_error += self.error(target_matrix[:,f:t],outputs)
                self.backward_propagate_error(target_matrix[:,f:t])
                self.update_weights()
            if verbose:
                print(f'> epoch={epoch+1}, lrate={self.l_rate:.3}, error={sum_error/m:.5f}')
        if verbose:
            print("Trained Weights:")
            print(*self.weights,sep="\n")
            
    def predict(self, inputs):
        outputs = self.forward_propagate(inputs.T).T.argmax(axis=-1)
        return outputs

    def score(self,X_test,y_test):
        return (self.predict(X_test)==y_test).mean()

#### Part 2 - Loading and processing dataset

In [3]:
titanic_df = pd.read_csv("datasets/titanic_processed.csv")
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
split_ratio = 0.8
s = int(split_ratio*X.shape[0])
X_train, X_test = X[:s],X[s:]
y_train, y_test = y[:s],y[s:]

#### Part 3 - Implementing BPNN

In [4]:
b = BPNN(
    hidden_layer_sizes = [5,3],
    l_rate=.6, n_epoch=20, batch_size=25,random_state=10
)
b.fit(X_train,y_train,verbose=True)
print("Evaluation:")
print("Acuracy of the classifier: ",b.score(X_test,y_test))

Initial Weights:
[[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
  0.19806286 0.76053071 0.16911084]
 [0.08833981 0.68535982 0.95339335 0.00394827 0.51219226 0.81262096
  0.61252607 0.72175532 0.29187607]
 [0.91777412 0.71457578 0.54254437 0.14217005 0.37334076 0.67413362
  0.44183317 0.43401399 0.61776698]
 [0.51313824 0.65039718 0.60103895 0.8052232  0.52164715 0.90864888
  0.31923609 0.09045935 0.30070006]
 [0.11398436 0.82868133 0.04689632 0.62628715 0.54758616 0.819287
  0.19894754 0.8568503  0.35165264]]
[[0.75464769 0.29596171 0.88393648 0.32551164 0.1650159  0.39252924]
 [0.09346037 0.82110566 0.15115202 0.38411445 0.94426071 0.98762547]
 [0.45630455 0.82612284 0.25137413 0.59737165 0.90283176 0.53455795]]
[[0.59020136 0.03928177 0.35718176 0.07961309]
 [0.30545992 0.33071931 0.7738303  0.03995921]]
Training:
> epoch=1, lrate=0.6, error=0.50494
> epoch=2, lrate=0.6, error=0.49556
> epoch=3, lrate=0.6, error=0.48575
> epoch=4, lrate=0.6, error=0.48282
> epoch

## b) Support Vector Machine

### Program 1 - Implementing SVM from scratch using CVXOPT

#### AIM
To implement SVM from scratch using CVXOPT.

#### Formula
##### SVM statement in dual form
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{aligned}
    & \min_{\vect{\alpha}}  \frac{1}{2}  \vect{\alpha}^T  \left( \vect{y}\vect{y}^T \circ (\Phi(\vect{X})\Phi(\vect{X})^T \right)  \vect{\alpha} - \vect{1}^T \vect{\alpha}\\
s.t.&  - \alpha_i \leq 0 \\
    & \alpha_i \leq C\\
    & y^T \vect{\alpha} = 0  
\end{aligned}
$$

##### Standard form of a quadratic program
$$
\begin{aligned}
    & \min_{\vect{x}}  \frac{1}{2}  \vect{x}^T  \vect{P}  \vect{x} - \vect{q}^T\vect{x}\\
s.t.&  \vect{G}\vect{x} \leq \vect{h} \\
    & A \vect{x} = \vect{b} 
\end{aligned}
$$

##### Formulization
$$
\begin{align*}
\vect{P}&= \vect{y}\vect{y}^T \circ \Phi(\vect{X})\Phi(\vect{X})^T
& \vect{q}&= - \vect{1}_n\\
\vect{G}&= \begin{bmatrix}
    \vect{-I}_n\\
    \vect{I}_n
\end{bmatrix} 
& \vect{h}&=\begin{bmatrix}
    C \cdot \vect{1}_n^T\\
    \vect{0}_n^T
\end{bmatrix}\\
\vect{A} &= [\vect{y}] 
&\vect{b} &= [0]\\
\end{align*}
$$

##### Decision Rule
$$
\vect{\hat y} = \mathrm{sign}\left( (\vect{\alpha}^T \circ \vect{y}^T) \Phi(\vect{X})\Phi(\vect{X})^T  + b \right) 
$$

#### Part 1 - Defining class for SVM

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from cvxopt import solvers
from cvxopt import matrix

from scipy.spatial.distance import cdist

In [6]:
class SVM:
    def __init__(self,C,kernel):
        self.C = C
        self.kernel = kernel

    def fit(self,X_train,y_train):
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(X_train)
        self.y = y_train.reshape(-1,1)
        
        n=self.X.shape[0]
        I_n = np.eye(n)
        P=(self.y@self.y.T)*self.kernel(self.X,self.X)
        q=np.full(n,-1)
        G=np.vstack((I_n,-1*I_n))
        h=np.hstack((np.full(n,self.C),np.zeros(n)))
        A=y_train.reshape(1,-1)
        b=np.zeros(1)

        P,q,G,h,A,b = map(lambda x : matrix(x,tc="d"),(P,q,G,h,A,b))

        solution = solvers.qp(P, q, G, h, A, b)
        self.a = np.asarray(solution['x']).squeeze()
        
        support_indices = np.logical_and(self.a>=1e-10, self.a<self.C)
        X_S = self.X[support_indices]
        self.b = np.mean(self.y - self.a*self.y.T @ self.kernel(self.X, X_S))

    def predict(self,X_test):
        X_test=self.scaler.transform(X_test)
        return np.sign(self.a*self.y.T @ self.kernel(self.X, X_test) + self.b)


#### Part 2 - Defining Radial Basis Function(RBF) Kernel

In [7]:
def rbf_kernel(X1,X2,sigma):
    return np.exp(-cdist(X1, X2, 'sqeuclidean') / (2*sigma**2))

#### Part 3 - Loading and Processing Dataset

In [8]:
titanic_df = pd.read_csv('datasets/titanic_processed.csv')
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
y[y==0] = -1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Part 4 - Implementing SVM

In [9]:
from functools import partial 
C = .35
sigma = .5
kernel = partial(rbf_kernel,sigma=sigma)

svm_classifier = SVM(C,kernel)
svm_classifier.fit(X_train,y_train)
y_pred = svm_classifier.predict(X_test)
print(f'Accuracy of the classifer: {(y_test == y_pred).mean()*100:.2f}%')


     pcost       dcost       gap    pres   dres
 0: -2.0672e+02 -6.0489e+02  6e+03  8e+00  1e-15
 1: -1.0821e+02 -5.1411e+02  6e+02  3e-01  1e-15
 2: -1.0989e+02 -1.6099e+02  5e+01  9e-03  2e-15
 3: -1.2250e+02 -1.3135e+02  9e+00  1e-03  1e-15
 4: -1.2441e+02 -1.2830e+02  4e+00  4e-04  1e-15
 5: -1.2525e+02 -1.2680e+02  2e+00  1e-04  1e-15
 6: -1.2558e+02 -1.2625e+02  7e-01  7e-06  1e-15
 7: -1.2572e+02 -1.2596e+02  2e-01  9e-07  1e-15
 8: -1.2577e+02 -1.2588e+02  1e-01  4e-07  1e-15
 9: -1.2578e+02 -1.2586e+02  8e-02  2e-07  9e-16
10: -1.2580e+02 -1.2582e+02  2e-02  2e-08  1e-15
11: -1.2581e+02 -1.2581e+02  6e-03  6e-09  1e-15
12: -1.2581e+02 -1.2581e+02  3e-03  1e-09  1e-15
13: -1.2581e+02 -1.2581e+02  9e-04  3e-10  9e-16
14: -1.2581e+02 -1.2581e+02  3e-04  9e-11  1e-15
15: -1.2581e+02 -1.2581e+02  2e-05  4e-15  1e-15
Optimal solution found.
Accuracy of the classifer: 73.03%
